#Introduction
Welcome to the first remote Eki hackaton. The aim of the hackaton is to build your first neural network and a learning scheme in a efficient way.The task is image classification on CIFAR10

**************
The code contains :

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

**************
Some **questions** too keep in mind :

    - which kind of data transformation and augmentation?
    - How building my neural network?
    - which kind of optimization?

Some tips here :) : [ML Lunch](https://bizoffice61.sharepoint.com/:p:/r/sites/Ekimetrics51/_layouts/15/Doc.aspx?sourcedoc=%7BA44BD8EB-39E2-4113-9861-A817680A7F55%7D&file=20200304%20-%20ML%20Lunch%20-%20Understanding%20Deep%20Learning.pptx&action=edit&mobileredirect=true)

**************
How to win?

The winner will propose a script which:

reachs at least 60% of accuracy on the dataset
builds a network with a minimum number of parameters.
Example :

People 1 : Accuracy : 58%, number of parameter : 10K

People 2 : Accuracy : 61%, number of parameters : 15K

People 3: Accuracy : 67%, number of paremeters : 16K

People 2 wins !

#Import all the libraries

In [6]:
!nvidia-smi

Thu Oct 15 23:09:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 2070   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   56C    P0    43W /  N/A |    857MiB /  8192MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [ ]:
dir(tf.test)

In [ ]:
tf.test.is_built_with_xla()

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
!nvidia-smi

In [ ]:
# import torch
# import torchvision
# import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
import requests

# Put the token you received in a mail here ! 
token = '5e9ed8b402f6580017600bca'

## Import own library

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split



---



# Load and normalizing the CIFAR10 training and test datasets using ``torchvision``

In [ ]:
########################################################################
# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose(
     [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



In [ ]:
import _pickle as pickle
def unpickle(file):

    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding ='bytes')
    X = dict[b'data']
    Y = dict[b'labels']
    fo.close()
    return X, Y

X, Y = unpickle('data/cifar-10-batches-py/test_batch')

tr = [
    'data_batch_1',
    'data_batch_2',
    'data_batch_3',
    'data_batch_4',
    'data_batch_5', 
]

te = [
    'test_batch'
]

X, Y = unpickle('data/cifar-10-batches-py/data_batch_1')

for elt in tr[1:]:
    X_temp, Y_temp = unpickle('data/cifar-10-batches-py/'+str(elt))
    X = np.concatenate([X, X_temp], axis = 0)
    Y+=Y_temp

X_test, Y_test = unpickle('data/cifar-10-batches-py/test_batch')

def reshape(x):
    red = x[:1024].reshape(32,32,1)
    green = x[1024:2048].reshape(32,32,1)
    blue = x[2048:3072].reshape(32,32,1)
    return np.concatenate([red, green, blue], axis = -1)

def batch_reshape(X):
    X1 = list(np.zeros(X.shape[0]))
    for i, elt in enumerate(X):
        X1[i] = reshape(elt)
    return X1

X = np.array(batch_reshape(X))
X_test = np.array(batch_reshape(X_test))

plt.imshow(X[5])

## Formatting into train and val

In [ ]:
X = X/255 - 0.5
X_test = X_test/255 -0.5

In [ ]:
y = np_utils.to_categorical(Y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

## Let's train at first a big model

In [ ]:
from tensorflow.keras.layers import Input,Concatenate, Conv2D, Lambda,Flatten, Activation, Dense, BatchNormalization, MaxPooling2D,AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam
import tensorflow as tf
import tensorflow.keras.backend as K

In [ ]:
def build_model():
    inputs = Input(shape = (32,32,3))

    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(10)(x)

    outputs = Activation('softmax')(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

optimizer = SGD(0.1)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=6, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)
reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                     mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

batch_size = 64
epochs = 40

history = model.fit_generator(aug.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs, callbacks = [stop, reduce])

In [ ]:
model.save_weights('model_big_normalized.h5')

In [ ]:
correct = 0
total = len(Y_test)

pred = np.argmax(model.predict(X_test), axis = 1).astype(int)

for i, elt in enumerate(pred):
    if elt == Y_test[i]:
        correct+=1


accuracy = 100 * correct / total

def num_weights(model):
    weights = 0
    for elt in model.trainable_weights:
        a = 1
        for elt1 in elt.shape:
            a*=elt1
        weights+=a
    return weights

trainable_parameters = num_weights(model)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    accuracy))

print("number of trainable parameters : %d"%trainable_parameters)

score = accuracy if accuracy <= 60. else 60. + 40 * np.tanh(10000/trainable_parameters)
print(score)

# requests.put(
#   'http://eki-hackathon.herokuapp.com/submitScoreAndMetrics', 
#   json={
#     'id': token,
#     'score': score,
#     'metrics': {"trainable parameters": trainable_parameters,
#                 "accuracy" : accuracy}
#   })

### Some attempt at distillation

In [ ]:
model_big = build_model()
model_big.load_weights('model_big_normalized.h5')
inputs = model_big.input
outputs = model_big.layers[-2].output

In [ ]:
teacher = Model(inputs, outputs)

train_logit = teacher.predict(X_train)
test_logit = teacher.predict(X_val)

y_traind = np.concatenate([y_train, train_logit], axis = 1)
y_vald = np.concatenate([y_val, test_logit], axis = 1)

##### Best distilled model stable 8826 weights int 255 images

In [ ]:
## Best stable
# temperature =  5

# inputs = Input(shape = (32,32,3))

# x = Conv2D(16, kernel_size=(3, 3), activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(16, kernel_size=(3, 3), activation='relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Flatten()(x)
# # x = Dense(256, activation='relu')(x)
# logits = Dense(10)(x)

# probabilities = Activation('softmax')(logits)
# # softed probabilities
# logits_T = Lambda(lambda x: x/temperature)(logits)
# probabilities_T = Activation('softmax')(logits_T)

# output = Concatenate()([probabilities, probabilities_T])


# distilled = Model(inputs, output)

##### Best Model unstable 6268 normalized images (might depend on training)

In [ ]:
temperature =  5

inputs = Input(shape = (32,32,3))

x = Conv2D(15, kernel_size=(3, 3),activation='relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(15, kernel_size=(3, 3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(21, kernel_size=(3, 3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(22, kernel_size=(3, 3),padding = 'same', activation='relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
logits = Dense(10)(x)

probabilities = Activation('softmax')(logits)
# softed probabilities
logits_T = Lambda(lambda x: x/temperature)(logits)
probabilities_T = Activation('softmax')(logits_T)

output = Concatenate()([probabilities, probabilities_T])


distilled = Model(inputs, output)


In [ ]:
distilled.summary()

In [ ]:
from tensorflow.keras.losses import categorical_crossentropy as logloss
from tensorflow.keras.metrics import categorical_accuracy
def knowledge_distillation_loss(y_true, y_pred, lambda_const):    
    
    # split in 
    #    onehot hard true targets
    #    logits from xception
    y_true, logits = y_true[:, :10], y_true[:, 10:]
    
    # convert logits to soft targets
    y_soft = K.softmax(logits/temperature)
    
    # split in 
    #    usual output probabilities
    #    probabilities made softer with temperature
    y_pred, y_pred_soft = y_pred[:, :10], y_pred[:, 10:]    
    
    return lambda_const*logloss(y_true, y_pred) + logloss(y_soft, y_pred_soft)

def accuracy(y_true, y_pred):
    y_true = y_true[:, :10]
    y_pred = y_pred[:, :10]
    return categorical_accuracy(y_true, y_pred)

In [ ]:
lambda_const = 0.2

distilled.compile(
    optimizer=SGD(lr=0.1, momentum=0.9, nesterov=True), 
    loss=lambda y_true, y_pred: knowledge_distillation_loss(y_true, y_pred, lambda_const), 
    metrics=[accuracy]
)

In [ ]:
stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=6, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)
reduce =tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                     mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001)

batch_size = 64
epochs = 40

history = distilled.fit_generator(aug.flow(X_train, y_traind, batch_size=batch_size),
    validation_data=(X_val, y_vald), steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs, callbacks = [stop, reduce])



In [ ]:
correct = 0
total = len(Y_test)

pred = np.argmax(distilled.predict(X_test)[:,:10], axis = 1).astype(int)

for i, elt in enumerate(pred):
    if elt == Y_test[i]:
        correct+=1


accuracy = 100 * correct / total

def num_weights(model):
    weights = 0
    for elt in model.trainable_weights:
        a = 1
        for elt1 in elt.shape:
            a*=elt1
        weights+=a
    return weights

trainable_parameters = num_weights(distilled)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    accuracy))

print("number of trainable parameters : %d"%trainable_parameters)

score = accuracy if accuracy <= 60. else 60. + 40 * np.tanh(10000/trainable_parameters)
print(score)

# requests.put(
#   'http://eki-hackathon.herokuapp.com/submitScoreAndMetrics', 
#   json={
#     'id': token,
#     'score': score,
#     'metrics': {"trainable parameters": trainable_parameters,
#                 "accuracy" : accuracy}
#   })